In [1]:
import pandas as pd
import geopandas 
import fiona
import matplotlib as plt
import folium

# Jakie emocje dominują w piosenkach w różnych krajach

In [2]:
spotify_final_database = pd.read_csv('Final database.csv')

C:\Users\kinga\AppData\Local\Temp\ipykernel_2732\1388054030.py:1: DtypeWarning: Columns (7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  spotify_final_database = pd.read_csv('Final database.csv')


In [3]:
songs_emotions = spotify_final_database[['Popularity', 'Country', 'Title', 'Artist', 'syuzhet_norm']]

### Żeby sprawdzić, czy w danym kraju domunują piosenki o negatywnie czy pozytywnie zabarwione emocjonalnie policzę sumę kolumny syuzhet_norm dla każdego z krajów 

In [4]:
#grupowanie danych po kraju 
emotions = songs_emotions.dropna()

emotions


,Popularity,Country,Title,Artist,syuzhet_norm
16,418.00,Global,it wont kill ya,The Chainsmokers - Louane,-1.0
17,581.20,USA,it wont kill ya,The Chainsmokers - Louane,-1.0
18,120.80,UK,it wont kill ya,The Chainsmokers - Louane,-1.0
19,404.00,Austria,it wont kill ya,The Chainsmokers - Louane,-1.0
20,29.60,Australia,it wont kill ya,The Chainsmokers - Louane,-1.0
...,...,...,...,...,...
170516,102.00,Taiwan,thank you,Meghan Trainor - R. City,-1.0
170564,631.60,Taiwan,somebody,Patrick Brasca,1.0
170577,1281.60,Taiwan,moonlight,Will Pan - Tia Ray,-1.0
170603,6.40,Taiwan,audio,Sia - Diplo - Labrinth - LSD,-1.0


In [5]:
country_emotions = pd.DataFrame(emotions.groupby('Country')['syuzhet_norm'].sum())
country_emotions

,syuzhet_norm
Country,
Argentina,106.0
Australia,127.0
Austria,95.0
Belgium,137.0
Brazil,107.0
Canada,-147.0
Chile,130.0
Colombia,117.0
Costa Rica,72.0


In [6]:
c_emotions = pd.read_csv('emotions.csv')

In [7]:
c_emotions

,Country,Emotions
0,Argentina,106.0
1,Australia,127.0
2,Austria,95.0
3,Belgium,137.0
4,Brazil,107.0
5,Canada,-147.0
6,Chile,130.0
7,Colombia,117.0
8,Costa Rica,72.0
9,Denmark,185.0


In [8]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
cities = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))

In [9]:
c_e=world.merge(c_emotions, how = 'left', left_on=['name'], right_on=['Country'])

In [10]:
c_e.name.unique()

array(['Fiji', 'Tanzania', 'W. Sahara', 'Canada',
       'United States of America', 'Kazakhstan', 'Uzbekistan',
       'Papua New Guinea', 'Indonesia', 'Argentina', 'Chile',
       'Dem. Rep. Congo', 'Somalia', 'Kenya', 'Sudan', 'Chad', 'Haiti',
       'Dominican Rep.', 'Russia', 'Bahamas', 'Falkland Is.', 'Norway',
       'Greenland', 'Fr. S. Antarctic Lands', 'Timor-Leste',
       'South Africa', 'Lesotho', 'Mexico', 'Uruguay', 'Brazil',
       'Bolivia', 'Peru', 'Colombia', 'Panama', 'Costa Rica', 'Nicaragua',
       'Honduras', 'El Salvador', 'Guatemala', 'Belize', 'Venezuela',
       'Guyana', 'Suriname', 'France', 'Ecuador', 'Puerto Rico',
       'Jamaica', 'Cuba', 'Zimbabwe', 'Botswana', 'Namibia', 'Senegal',
       'Mali', 'Mauritania', 'Benin', 'Niger', 'Nigeria', 'Cameroon',
       'Togo', 'Ghana', "Côte d'Ivoire", 'Guinea', 'Guinea-Bissau',
       'Liberia', 'Sierra Leone', 'Burkina Faso', 'Central African Rep.',
       'Congo', 'Gabon', 'Eq. Guinea', 'Zambia', 'Malawi', 'M

In [13]:
emotions_map = folium.Map()
choropleth = folium.Choropleth(
    geo_data=c_e,
    name="emotions_by_country",
    data=c_e,
    columns=["Country", "Emotions"],
    key_on="feature.properties.name",
    fill_color="RdYlGn",
    fill_opacity=0.8,
    line_opacity=0.4,
    legend_name="Emotions", 
).add_to(emotions_map)

folium.LayerControl(collapsed=True).add_to(emotions_map)
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(        
        fields=['Country'
                ,'Emotions'
               ],
        aliases=['Country: '
                ,'Emotions level: '
                 ],
        style=("background-color: white; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
emotions_map.save('emotions_map.html')

# Wniosek - w krajach oznaczonych na czerwono użytkownicy Spotify częściej słuchają piosenek, których ton tekstu jest negatywny, natomiast w krajach oznaczonych na zielono częściej słuchane są piosenki, kórych ton tekstu jest pozytywny